In [1]:
using LinearAlgebra

include("./MPS.jl")
using .MPSforQuantum

# MPS

### Left normalized

```mps = MPS(C0, eps)```  
C0: state vector  
eps: allowed error  

return: mps\[site\]\[physical\]

In [4]:
N = 20
C0 = normalize!(rand(ComplexF64, 2^N))
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3
mps = MPS(C0, D) # convert to MPS
mps_size(mps) # print the size of MPS

n = 0 # check the state |bin(n)>
println("Restore from MPS: ", restore(mps, n))
println("Original state: ", C0[n + 1])

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 8)
array 4's size: (8, 10)
array 5's size: (10, 10)
array 6's size: (10, 10)
array 7's size: (10, 10)
array 8's size: (10, 10)
array 9's size: (10, 10)
array 10's size: (10, 10)
array 11's size: (10, 10)
array 12's size: (10, 10)
array 13's size: (10, 10)
array 14's size: (10, 10)
array 15's size: (10, 10)
array 16's size: (10, 10)
array 17's size: (10, 8)
array 18's size: (8, 4)
array 19's size: (4, 2)
array 20's size: (2, 1)
Num of parameters: 2888
2^N: 1048576
Restore from MPS: 1.0 + 0.0im
Original state: 1.0 + 0.0im


In [6]:
mps[20][1] # size (1,1,2)

2×1 Array{Complex{Float64},2}:
 1.0 + 0.0im
 0.0 + 0.0im

### Right Normalized

In [5]:
N = 20
C0 = normalize!(rand(ComplexF64, 2^N))
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '010'

D = 10
eps = 1e-3
mps = MPS(C0, D, 'r') # convert to MPS
mps_size(mps) # print the size of MPS

n = 0 # check the state |bin(n)>
println("Restore from MPS: ", restore(mps, n))
println("Original state: ", C0[n + 1])

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 8)
array 4's size: (8, 10)
array 5's size: (10, 10)
array 6's size: (10, 10)
array 7's size: (10, 10)
array 8's size: (10, 10)
array 9's size: (10, 10)
array 10's size: (10, 10)
array 11's size: (10, 10)
array 12's size: (10, 10)
array 13's size: (10, 10)
array 14's size: (10, 10)
array 15's size: (10, 10)
array 16's size: (10, 10)
array 17's size: (10, 8)
array 18's size: (8, 4)
array 19's size: (4, 2)
array 20's size: (2, 1)
Num of parameters: 2888
2^N: 1048576
Restore from MPS: 1.0 + 0.0im
Original state: 1.0 + 0.0im


In [45]:
typeof('l')

Char

### Inner product between same states is Identity matrix
each site is normalized,  
$\sum_{i}A_{i}^{\dagger} A_{i} = I$

In [87]:
I_ = mps[3][1]' * mps[3][1] + mps[3][2]' * mps[3][2]
diag(I_)

1-element Array{Complex{Float64},1}:
 1.0 + 0.0im

## 簡単な例
行列```tmp```を特異値分解により3つの行列に分ける。  
3つの行列をさらに2つの行列に分けて(=physical index)、physical indexで指定された片割れ同士で3つの行列をかけると分解前の行列の要素が計算できることを示す。

In [3]:
tmp = svd(reshape([1, 2, 3, 4, 5, 6, 7, 8], 2, 4))
A1 = tmp.U
C1 = diagm(tmp.S) * tmp.Vt
println(transpose(tmp.U[1, :]) * diagm(tmp.S) * tmp.Vt)
println(transpose(tmp.U[2, :]) * diagm(tmp.S) * tmp.Vt)

[1.0000000000000007 3.0 5.000000000000002 7.000000000000001]
[2.000000000000006 4.0000000000000036 6.000000000000005 8.000000000000007]


In [4]:
tmp = svd(reshape(C1, 4, 2))
A2 = tmp.U
A3 = diagm(tmp.S) * tmp.Vt

2×2 Array{Float64,2}:
  5.3519   13.1824
 -1.16498   0.472968

In [5]:
println(A1)
println(A2)
println(A3)

[-0.6414230279950722 -0.7671873950721774; -0.7671873950721774 0.6414230279950723]
[-0.5661645853690824 -0.7332805685413178; 0.014454499999906334 -0.3762291024141132; -0.8241208487575695 0.49992875081918764; -0.00857649967705636 -0.2661249976998753]
[5.351898726900281 13.182423969136662; -1.1649806938328169 0.4729675442683846]


In [91]:
transpose(A1[1, :]) * A2[1:2, :] * transpose(A3)[1, :] # '000'

1.0000000000000002

# MPO

期待値を求めたいハミルトニアン  
簡単のため、3 siteの系を考える  
$Z_1 Z_2 + Z_2 Z_3$

MPOは下記のように組む。  
site数と同じ3つの行列をする。  
行列同士の積を計算すると、上記のハミルトニアンになるように要素を決めている。

$$
\left[\begin{array}{c}
    Z_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0\\
    0 & Z_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3\\
    Z_3
\end{array}\right] \quad
$$

In [92]:
function dstack(A::Array{ComplexF64,2}, B::Array{ComplexF64,2})
    return cat(A, B, dims = 3)
end

function ddstack(A::Array{ComplexF64,2}, B::Array{ComplexF64,2}, 
                C::Array{ComplexF64,2}, D::Array{ComplexF64,2})
    AC = cat(A, C, dims = 3)
    BD = cat(B, D, dims = 3)
    return cat(AC, BD, dims = 4)
end

ddstack (generic function with 1 method)

In [93]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack(pauliZ, pauliI))
push!(O, ddstack(pauliZ, zero, zero, pauliZ))
push!(O, dstack(pauliI, pauliZ))
#O = [[pauliZ pauliI] [pauliZ zero;pauliZ zero] [pauliI pauliZ]]
# O[site], [phys1, phys2, i, j]

O[1]

2×2×2 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

# General inner products
2つのMPSにそれぞれ異なる量子ゲートをかけてから内積を計算する

In [7]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))
pauliX = convert(Array{ComplexF64,2}, [0 1; 1 0])

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  1.0+0.0im
 1.0+0.0im  0.0+0.0im

In [95]:
N = 3
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'
# C0[8] = 1 # '111'

D = 10
eps = 1e-3
mps1 = MPS(C0, eps)
mps1 = OneQubitGate(mps1, Hadamard, 0)
mps1 = OneQubitGate(mps1, Hadamard, 1)
mps1 = OneQubitGate(mps1, Hadamard, 2)

mps2 = MPS(C0, eps)
mps2 = OneQubitGate(mps2, pauliX, 0)
mps1

3-element Array{Any,1}:
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]

In [96]:
function inner_product(arrs1::Array{Any, 1}, arrs2::Array{Any, 1})
    N = Int64(size(arrs1)[1])
    ip = arrs1[1][1]' * arrs2[1][1] + arrs1[1][2]' * arrs2[1][2]
    for i = 2:N
        phys0 = arrs1[i][1]' * ip * arrs2[i][1]
        phys1 = arrs1[i][2]' * ip * arrs2[i][2]
        ip = phys0 + phys1
    end
    return ip
end

inner_product (generic function with 1 method)

In [97]:
inner_product(mps1, mps2)

1×1 Array{Complex{Float64},2}:
 0.3535533905932737 + 0.0im

### Bell state

In [98]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))

N = 20
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3

mps1 = MPS(C0, eps)
println("Initial MPS")
mps1 = OneQubitGate(mps1, Hadamard, 0) # -> '001' & '000'
mps_size(mps1)

arr = CX(mps1, 0, eps) # 今の所、i と i+1の間での計算のみ。 多分swapゲートが必要
println("\nBell state MPS")
mps_size(arr)
println()
for i=0:3
    res = restore(arr, i) # MPS で表された状態から、| bin(i) > の係数を計算する関数
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

Initial MPS
array 1's size: (1, 1)
array 2's size: (1, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1, 1)
Num of parameters: 40
2^N: 1048576

Bell state MPS
array 1's size: (1, 2)
array 2's size: (2, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1

## 確認
状態ベクトルでBell state作ってからMPSに変換して同じになってるか確認してみた。

In [99]:
N = 2
C0 = zeros(ComplexF64, 2^N)
C0[1] = 0.7071067811865475
C0[4] = 0.7071067811865475

eps = 1e-3

mps1 = MPS(C0, eps)
mps_size(mps1)
for i=0:3
    res = restore(mps1, i)
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

array 1's size: (1, 2)
array 2's size: (2, 1)
Num of parameters: 8
2^N: 4
|00>: 0.7071067811865475 + 0.0im
|01>: 0.0 + 0.0im
|10>: 0.0 + 0.0im
|11>: 0.7071067811865475 + 0.0im


# Expectation value

In [8]:
# prepare bell state
N = 3
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3

mps0 = MPS(C0, eps)
mps = OneQubitGate(mps0, Hadamard, 0) # -> '001' & '000'　: indexは後ろから数える
#mps = OneQubitGate(mps, Hadamard, 1)
mps = OneQubitGate(mps, Hadamard, 2)

mps = CX(mps, 0, eps)

for i=1:2, j=1:2, k=1:2
    println("$(i-1)$(j-1)$(k-1): ", mps[1][k] * mps[2][j] * mps[3][i])
end

000: Complex{Float64}[0.4999999999999999 + 0.0im]
001: Complex{Float64}[0.0 + 0.0im]
010: Complex{Float64}[0.0 + 0.0im]
011: Complex{Float64}[0.4999999999999999 + 0.0im]
100: Complex{Float64}[0.4999999999999999 + 0.0im]
101: Complex{Float64}[0.0 + 0.0im]
110: Complex{Float64}[0.0 + 0.0im]
111: Complex{Float64}[0.4999999999999999 + 0.0im]


$$
\left[\begin{array}{c}
    Z_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0\\
    0 & Z_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3\\
    Z_3
\end{array}\right] \quad
$$

In [114]:
mps[2] # mps [site][physical]

2-element Array{Array{Complex{Float64},2},1}:
 [0.7071067811865475 + 0.0im; 0.0 + 0.0im]
 [0.0 + 0.0im; 0.7071067811865475 + 0.0im]

各siteでまずMPSx2,MPOをcontructする。  

$$
F^{[i]}_{a_i,b_i,a'_i} = \sum_{\sigma_{i},a_{i-1}}(A^{[i]\sigma_i\dagger})_{a_i,a_{i-1}}
\biggl(\sum_{\sigma'_{i},b_{i-1}}W^{[i]\sigma_i \sigma'_i}_{b_{i-1},b_i} 
\biggl(\sum_{a'_{i-1}}F^{[i-1]}_{a_{i-1},b_{i-1},a'_{i-1}} A^{[i]\sigma'_i}_{a'_{i-1},a'_i} \biggr) \biggr)
$$

In [6]:
using TensorOperations

In [118]:
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '010'
mps = MPS(C0, eps)
println(size(mps[3][1]))
# AA = cat(mps[1][1][1], mps[1][2][1], dims=2)
#  AA_dag = cat(mps[1][1]'[1], mps[1][2]'[1], dims=2)
mps[3][1][1]

(1, 1)


1.0 + 0.0im

In [151]:
function expectation_(mps::Array{Any,1}, O::Array{Any,1})
    N_site = size(mps)[1]
    contracted_sites = []
    n_phys = 2 #ss
    for i=1:N_site
        if i==1
            # first step
            tmp_mps = cat(mps[i][1][1], mps[i][2][1], dims=2)
            tmp_mps_dag = cat(mps[i][1]'[1], mps[i][2]'[1], dims=2)
            tmp_O = O[i]
            a_len = size(mps[i][1])[2]
            b_len = size(O[i][1, 1, :])[1] # bb0
            arr_0 = zeros(ComplexF64, n_phys, b_len, a_len)
            @tensor begin
                arr_0[sigma1, b, a_] = tmp_O[sigma1, sigma2, b] * tmp_mps[a_, sigma2]
            end
            arr = zeros(ComplexF64, a_len, b_len, a_len)
            @tensor begin
                arr[a, b, a_] = tmp_mps_dag[a, sigma1] * arr_0[sigma1, b, a_]
            end

        elseif i==N_site
            # last step
            tmp_mps = cat(mps[i][1][1], mps[i][2][1], dims=3)
            tmp_mps_dag = cat(mps[i][1]'[1], mps[i][2]'[1], dims=3)
            tmp_O = O[i]
            tmp_site = contracted_sites[i-1]
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len2 = size(O[i][1, 1, :])[1]
            arr_0 = zeros(ComplexF64, n_phys, a_len1, b_len2, a_len2)
            @tensor begin
                arr_0[sigma2, a1, b2, a2] = tmp_site[a1, b2, a1_0] * tmp_mps[a1_0, a2, sigma2]
            end
            arr_1 = zeros(ComplexF64, n_phys, a_len1, a_len2)
            @tensor begin
                arr_1[sigma1, a1, a2] = tmp_O[sigma1, sigma2, b2] * arr_0[sigma2, a1, b2, a2]
            end

            @tensor begin
                arr = tmp_mps_dag[a2, a1, sigma1] * arr_1[sigma1, a1, a2]
            end

        else
            # Middle step
            tmp_mps = cat(mps[i][1], mps[i][2], dims=3)
            tmp_mps_dag = cat(mps[i][1]', mps[i][2]', dims=3)
            tmp_O = O[i]
            tmp_site = contracted_sites[i-1]
            a_len1 = size(mps[i-1][1])[2]
            a_len2 = size(mps[i][1])[2]
            b_len1, b_len2 = size(O[i][1, 1, :, :])
            arr_0 = zeros(ComplexF64, n_phys, a_len1, b_len1, a_len2)
            @tensor begin
                arr_0[sigma2, a1, b1, a2] = tmp_site[a1, b1, a0_0] * tmp_mps[a0_0, a2, sigma2]
            end

            arr_1 = zeros(ComplexF64, n_phys, b_len2, a_len1, a_len2)
            @tensor begin
                arr_1[sigma1, b2, a1, a2] = tmp_O[sigma1, sigma2, b1, b2] * arr_0[sigma2, a1, b1, a2]
            end

            arr = zeros(ComplexF64, a_len2, b_len2, a_len2)
            @tensor begin
                arr[a2, b2, a2_] = tmp_mps_dag[a2, a1, sigma1] * arr_1[sigma1, b2, a1, a2_]
            end
        end
        push!(contracted_sites, arr)
    end
    return contracted_sites[N_site]
end


expectation_ (generic function with 1 method)

$Z_1 Z_2 + Z_2 Z_3$

In [152]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI)))
push!(O, ddstack([(pauliZ, zero), (zero, pauliZ)]))
push!(O, dstack((pauliI, pauliZ)))

3-element Array{Any,1}:
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

In [153]:
N = 3
eps = 1e-3
D = 10

for i in 1:8
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # '010'
    mps = MPS(C0, eps)
    expc = expectation_(mps, O)
    println(expc)
end

2.0 + 0.0im
0.0 + 0.0im
-2.0 + 0.0im
0.0 + 0.0im
0.0 + 0.0im
-2.0 + 0.0im
0.0 + 0.0im
2.0 + 0.0im


In [303]:
s = bitstring(2)[end - (3):end]

"0010"